# Sistema RAG: PLN com Embedding em VectorDB

Este notebook implementa um sistema de Retrieval-Augmented Generation (RAG) que permite fazer perguntas sobre documentos PDF.

**Fluxo completo:** PDF → Ler → PLN → Chunks → Embedding → VectorDB → Pesquisa → Contexto

**Bibliotecas utilizadas:**
- **LangChain**: Framework para dividir textos em chunks (pedaços) menores e gerenciáveis
- **SentenceTransformers**: Converte texto em vetores numéricos (embeddings) para busca semântica
- **pdfplumber**: Extrai texto de arquivos PDF mantendo a formatação
- **spaCy**: Biblioteca avançada de PLN para tokenização, lematização e análise linguística
- **NLTK**: Fornece stopwords (palavras irrelevantes) em português
- **re**: Expressões regulares para limpeza de texto

In [102]:
# Para chunks e embeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter # pip install langchain
from sentence_transformers import SentenceTransformer # pip install sentence_transformers
# Para leitura de PDF
import pdfplumber # pip install pdfplumber
# Para tratamento de texto
import re
import spacy # python -m spacy download pt_core_news_sm
import nltk # pip install nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Bibliotecas para Banco de Dados Vetorial (Vector Database)

**O que é um VectorDB?**
Um banco de dados vetorial armazena representações numéricas (embeddings) de textos e permite buscar documentos similares usando distância vetorial ao invés de correspondência exata de palavras.

**Por que usar ChromaDB?**
- **Busca semântica**: Encontra documentos por significado, não apenas por palavras-chave
- **Rápido**: Otimizado para buscar entre milhões de vetores
- **Simples**: Fácil de usar e não requer configuração complexa
- **Em memória**: Perfeito para protótipos e aplicações pequenas/médias

In [103]:
# Bibliotecas para Banco de Dados Vetorial (Vector Database)
import chromadb
from langchain_chroma import Chroma

## ETAPA 1: Leitura e Extração de Texto do PDF

**Objetivo:** Extrair todo o conteúdo textual do arquivo PDF para processamento.

**Como funciona:**
1. O `pdfplumber` abre o arquivo PDF
2. Percorre todas as páginas extraindo o texto
3. Remove quebras de linha para criar um texto contínuo

**Por que fazer isso?**
- PDFs armazenam texto de forma complexa (posicionamento, fontes, etc.)
- Precisamos de texto puro para aplicar PLN
- Quebras de linha podem interferir no processamento posterior

In [104]:
# PASSO 1: LEITURA DO PDF
# Função para extrair texto de todas as páginas do PDF

def ler_pdf(caminho_pdf):
    """Extrai texto de todas as páginas de um arquivo PDF"""
    leitor_pdf = pdfplumber.open(caminho_pdf)
    print(f'Quantidade de páginas: {len(leitor_pdf.pages)}')
    
    texto = ""
    total_linhas = 0
    
    # Percorre todas as páginas do PDF
    for pagina in range(len(leitor_pdf.pages)):
        texto_pagina = leitor_pdf.pages[pagina].extract_text()
        # Conta linhas na página (separadas por \n)
        linhas_pagina = texto_pagina.count('\n') + 1
        total_linhas += linhas_pagina
        texto += texto_pagina
    
    print(f'Total de linhas no PDF: {total_linhas}')

    # Remove quebras de linha para facilitar o processamento
    texto = texto.replace("\n", " ")
    return texto

In [105]:
# APLICANDO PASSO 1: Carregamento e exibição do conteúdo do PDF
arquivo_pdf = 'C:\\Development\\provaIA\\dataset\\buscainformada.pdf'
texto_pdf = ler_pdf(arquivo_pdf)

# Verificando o resultado da extração
print(f"PDF carregado: {len(texto_pdf)} caracteres")
print(f"Preview: {texto_pdf[:200]}...")


Quantidade de páginas: 1
Total de linhas no PDF: 30
PDF carregado: 2219 caracteres
Preview: BUSCA INFORMADA Diferentemente da Busca Exaustiva, onde não se sabe qual o melhor nó de fronteira a ser expandido, a Busca Heurística estima qual o melhor nó da fronteira a ser expandido com base em f...


## ETAPA 2: Configuração do PLN e Stopwords

**Justificativa Técnica Profunda:**
O PLN é crucial para **normalizar** o texto antes de gerar embeddings. Sem PLN, palavras como "correndo", "correr", "corre" gerariam embeddings diferentes, mesmo tendo o mesmo conceito. A lematização garante que todas sejam reduzidas a "correr", melhorando a **consistência semântica** e reduzindo a **dimensionalidade do vocabulário**.

Além disso, stopwords ("a", "de", "para") ocupam uma boa parte do texto mas contribuem quase nada para o significado. Removê-las:
- Reduz ruído nos embeddings
- Acelera o processamento
- Foca embeddings em palavras-chave relevantes

**O que são Stopwords?**
Palavras muito comuns que não agregam significado semântico ("a", "de", "para", "com", etc.). 

**Por que usar spaCy?**
- Modelo treinado especificamente para português (pt_core_news_sm)
- Realiza múltiplas tarefas em um único processamento (tokenização + lematização + POS tagging)
- Mais preciso que ferramentas simples (entende contexto linguístico)
- Lemmatização baseada em morfologia, não em regras heurísticas

In [106]:
# PASSO 2: CONFIGURAÇÃO DO PLN (Processamento de Linguagem Natural)

# Carregar o modelo de linguagem portuguesa do spaCy
nlp = spacy.load("pt_core_news_sm")

# Definir palavras irrelevantes (stopwords) que serão removidas
api_stop_words = set(stopwords.words('portuguese'))  # Stopwords padrão do NLTK
minhas_stop_words = {'a','e','i','o', 'u'}  # Vogais adicionais
stop_words = api_stop_words | minhas_stop_words  # União dos conjuntos

print(f"✓ Stopwords carregadas: {len(stop_words)} palavras")
print(f"✓ Stopwords carregadas: {(stop_words)} palavras")

✓ Stopwords carregadas: 209 palavras
✓ Stopwords carregadas: {'nossos', 'temos', 'hajamos', 'do', 'isso', 'tem', 'esta', 'só', 'tuas', 'nos', 'haver', 'era', 'estava', 'isto', 'há', 'aqueles', 'tivéssemos', 'pela', 'aquele', 'tiveram', 'aos', 'estar', 'esteja', 'houverá', 'estivermos', 'minha', 'não', 'esse', 'tiverem', 'tém', 'houvera', 'numa', 'essas', 'u', 'dos', 'seria', 'tenho', 'houvessem', 'fora', 'com', 'for', 'num', 'em', 'pelo', 'hão', 'os', 'fôramos', 'estes', 'ser', 'dele', 'seus', 'serei', 'vos', 'teríamos', 'entre', 'estamos', 'fosse', 'seja', 'de', 'seremos', 'ao', 'que', 'teus', 'i', 'nossas', 'tu', 'vocês', 'meus', 'meu', 'sejam', 'e', 'mais', 'estivéssemos', 'estivera', 'estejamos', 'fui', 'houvermos', 'para', 'houveram', 'hajam', 'aquilo', 'quem', 'fôssemos', 'por', 'serão', 'deles', 'suas', 'houverei', 'sua', 'me', 'éramos', 'forem', 'houvéssemos', 'nossa', 'depois', 'te', 'teriam', 'uma', 'nosso', 'mesmo', 'houvéramos', 'esteve', 'delas', 'está', 'muito', 'teremos'

### Função de Tratamento PLN: Lematização

**O que faz esta função?**
Aplica 5 técnicas de PLN para limpar e normalizar o texto:

1. **Normalização**: Converte tudo para minúsculas ("Casa" = "casa")
2. **Remoção de ruído**: Elimina números, pontuação e caracteres especiais
3. **Tokenização**: Divide o texto em palavras individuais (tokens)
4. **Remoção de stopwords**: Elimina palavras irrelevantes
5. **Lematização**: Reduz palavras à sua forma canônica (lema)

**IMPORTANTE: Lematização vs Stemming**

**Lematização (usado aqui):**
- Reduz palavras ao seu **lema** (forma base no dicionário)
- Exemplos: "correndo" → "correr", "melhores" → "bom", "crianças" → "criança"
- **Preserva o significado** da palavra
- Mais lento, mas muito mais preciso

**Stemming (NÃO usado):**
- Apenas corta o final das palavras
- Exemplos: "correndo" → "corr", "melhores" → "melhor"
- Pode **confundir o sentido**: "estudante" e "estudar" viram "estud"
- Mais rápido, mas menos preciso

**Por que Lematização é melhor para RAG?**
Como vamos usar embeddings semânticos, precisamos que as palavras mantenham seu significado real. O lema é o "cerne" da palavra, sua essência semântica, enquanto o stemming pode criar ambiguidade e prejudicar a busca.

In [107]:
# Função para fazer o tratamento de linguagem natural usando spaCy
# IMPORTANTE: Usa LEMATIZAÇÃO (não stemming) para preservar o significado das palavras
def tratamento_pln(texto):

    # 1. Normalização: Colocar o texto em minúsculas
    # JUSTIFICATIVA: Garante que "Machine", "machine" e "MACHINE" sejam tratadas como a mesma palavra
    # Sem isso, geraríamos 3 embeddings diferentes para o mesmo conceito, desperdiçando espaço vetorial
    # e reduzindo a precisão da busca (a pergunta "machine learning" não encontraria "Machine Learning")
    texto = texto.lower()

    # 2. Remoção de números, pontuações e caracteres especiais
    texto = re.sub(r'[^a-zA-Záéíóú\s]', '', texto) # na expressão regular estão as exceções

    # 3. Tokenização com spaCy
    doc = nlp(texto)
    tokens = [token.text for token in doc]

    # 4. Remoção de stopwords, remoção de pontuação
    # e Lematização (clean_tokens = tokens lematizados e sem stopwords)
    clean_tokens = [token.lemma_ for token in doc if token.text not in
    stop_words and not token.is_punct]

    # 5. Juntar tokens lematizados de volta em uma string
    clean_text = ' '.join(clean_tokens)

    return clean_text
    #return texto

In [108]:
# APLICANDO PASSO 2: Processamento de Linguagem Natural no texto
texto_pdf_tratado = tratamento_pln(texto_pdf)

# Comparando tamanhos antes e depois do processamento
reducao = len(texto_pdf) - len(texto_pdf_tratado)
print(f"✓ PLN aplicado: {len(texto_pdf)} → {len(texto_pdf_tratado)} caracteres (redução de {reducao})")
print(f"Preview tratado: {texto_pdf_tratado[:150]}...")

✓ PLN aplicado: 2219 → 1586 caracteres (redução de 633)
Preview tratado: busco informar diferentemente busca exaustivo onde saber bom nó fronteira expander busca heurístico estimo bom nó fronteira expander base fune heuríst...


## ETAPA 3: Criação de Chunks (Fragmentação)

**O que são Chunks?**
Pedaços pequenos de texto que podem ser processados e buscados independentemente.

**Justificativa Técnica Profunda:**
O chunking evita que o modelo de embeddings receba um texto que ultrapasse seu limite máximo de tokens (geralmente 256-512 tokens), e também **melhora a granularidade na recuperação semântica**:

- **Chunk grande demais** → Gera embeddings muito genéricos que capturam múltiplos tópicos, reduzindo a precisão da busca
- **Chunk pequeno demais** → Perde contexto suficiente para entender o significado completo, gerando fragmentos sem sentido
- **Chunk balanceado (150 chars)** → Mantém uma ideia completa sem misturar tópicos diferentes

**Por que dividir o texto em chunks?**
1. **Limitação de tamanho**: Modelos de embedding têm limite de tokens (~256-512 tokens)
2. **Precisão**: Chunks menores = busca mais precisa (um chunk = um conceito)
3. **Contexto**: Cada chunk mantém contexto local suficiente sem ruído
4. **Performance**: Busca vetorial é mais eficiente com vetores específicos

**Parâmetros importantes:**
- `chunk_size=150`: Tamanho máximo de cada chunk (em caracteres)

- `chunk_overlap=30`: Sobreposição entre chunks para não perder contexto nas bordasNote a sobreposição "é incrível. Usa" que mantém o contexto.

  - **Por que overlap?** Evita que uma frase importante seja "cortada ao meio" entre dois chunks

- Chunk 2: "é incrível. Usa algoritmos para aprender."

**Exemplo:**- Chunk 1: "O machine learning é incrível. Usa"
Texto: "O machine learning é incrível. Usa algoritmos para aprender."

In [109]:
# PASSO 3: CRIAÇÃO DE CHUNKS (Divisão do texto em pedaços menores)
# Dividir o texto em chunks para facilitar a busca semântica
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,      # Tamanho máximo de cada chunk
    chunk_overlap=30     # Sobreposição entre chunks para manter contexto
)

chunks = text_splitter.split_text(texto_pdf_tratado)

print(f"✓ Texto dividido em {len(chunks)} chunks")
print(f"Exemplo do primeiro chunk: {chunks[0][:100]}...")

✓ Texto dividido em 14 chunks
Exemplo do primeiro chunk: busco informar diferentemente busca exaustivo onde saber bom nó fronteira expander busca heurístico ...


## ETAPA 4: Geração de Embeddings

**O que são Embeddings?**
Representações numéricas (vetores) de textos que capturam seu significado semântico. Textos similares têm vetores próximos no espaço vetorial.

**Justificativa Técnica:**
O modelo `all-MiniLM-L6-v2` transforma cada chunk em um vetor de **384 números**. Mais dimensões = mais capacidade de capturar nuances. Com apenas 10 dimensões, "banco" (assento) e "banco" (instituição) ficariam próximos demais. Com 384 dimensões, o modelo separa contextos diferentes.

### Matemática: Como Medir Similaridade

**Cosine Similarity (Similaridade do Cosseno):**
```
similarity = (A · B) / (||A|| × ||B||)
```

Onde:
- `A · B` = produto escalar dos vetores
- `||A||` = tamanho do vetor A
- **Resultado:** -1 (opostos) a +1 (idênticos)

**Por que Cosine?**
- Mede o **ângulo** entre vetores, não distância absoluta
- Textos curtos e longos podem ser comparados de forma justa
- Foca na "direção semântica"

**Exemplo prático:**
- "cachorro" → [0.2, 0.8, 0.1, ..., 0.3] (384 números)
- "cão" → [0.19, 0.81, 0.09, ..., 0.29] → similarity ≈ **0.95** (muito similar!)
- "computador" → [0.7, 0.1, 0.9, ..., 0.6] → similarity ≈ **0.12** (bem diferente)

**Por que usar embeddings?**
- Busca por **significado**, não apenas palavras exatas
- "Como treinar modelos?" encontra "treinamento de algoritmos"
- Muito mais poderoso que busca tradicional por keywords

In [110]:
# PASSO 4: GERAÇÃO DE EMBEDDINGS (Conversão de texto em vetores numéricos)
# Carregar modelo que converte texto em vetores para busca semântica
model = SentenceTransformer('all-MiniLM-L6-v2')

# Converter cada chunk em um vetor numérico (embedding)
embeddings = model.encode(chunks)

print(f"✓ Gerados {len(embeddings)} embeddings de {len(embeddings[0])} dimensões")
print(f"Exemplo (primeiros 10 valores): {embeddings[0][:10]}")


✓ Gerados 14 embeddings de 384 dimensões
Exemplo (primeiros 10 valores): [ 0.1015088  -0.03895828 -0.02834271 -0.04518022 -0.02155408 -0.02779558
 -0.00316708  0.07924382 -0.025738    0.03808332]


In [111]:
# Gerando IDs automaticamente
uids = [f"doc_{i}" for i in range(len(chunks))]

## ETAPA 5: Armazenamento no VectorDB (ChromaDB)

**O que faz o ChromaDB?**
Armazena os embeddings de forma otimizada para busca rápida por similaridade.

**Justificativa Técnica:**
O VectorDB não é apenas um "banco de dados normal". Ele usa estruturas de dados especializadas (como HNSW - Hierarchical Navigable Small World) para encontrar vetores similares em tempo **logarítmico** O(log n), em vez de comparar com todos os vetores O(n). Isso permite buscar entre milhões de documentos em milissegundos.

**Estrutura dos dados:**
- `documents`: Textos originais dos chunks (para retornar ao usuário)
- `embeddings`: Vetores numéricos (384 dimensões cada) - usados na busca
- `ids`: Identificadores únicos (doc_0, doc_1, doc_2, ...) - para rastreamento

**Como funciona a busca?**
1. Você faz uma pergunta: "Como funciona machine learning?"
2. A pergunta é convertida em embedding (mesmo modelo: all-MiniLM-L6-v2)
3. ChromaDB calcula a **cosine similarity** entre o embedding da pergunta e todos os chunks
4. Retorna os chunks mais próximos (mais similares) - geralmente top 3 a 5

**Métrica de distância:**
Usa distância cosseno - quanto menor, mais similar:

- 0.0 = idênticos (cosine similarity = 1.0) **CUIDADO:** ChromaDB retorna **distance** (distância), não similarity. Por isso: menor = melhor!

- 0.5 = moderadamente similares (cosine similarity = 0.5)
- 1.0 = completamente diferentes (cosine similarity = 0.0)

In [112]:
# PASSO 5: CRIAÇÃO DO BANCO DE DADOS VETORIAL (VectorDB)
# Criar banco de dados vetorial para armazenar e buscar embeddings
client = chromadb.Client()

# Nome da collection
col_name = "machlrn"

# Se já existir, obter a collection; caso contrário, criar
existing_collections = [c.name for c in client.list_collections()]
if col_name in existing_collections:
    collection = client.get_collection(name=col_name)
else:
    collection = client.create_collection(name=col_name)

# Adicionar (ou atualizar) chunks, embeddings e IDs ao banco de dados
# Usar upsert se disponível para evitar erro com ids já existentes
if hasattr(collection, "upsert"):
    collection.upsert(
        documents=chunks,      # Textos originais
        embeddings=embeddings, # Vetores numéricos
        ids=uids               # Identificadores únicos
    )
else:
    # Fallback para add (pode falhar se ids já existirem)
    collection.add(
        documents=chunks,      # Textos originais
        embeddings=embeddings, # Vetores numéricos
        ids=uids               # Identificadores únicos
    )

print(f"✓ Banco vetorial criado/atualizado com {len(chunks)} documentos")

✓ Banco vetorial criado/atualizado com 14 documentos


In [113]:
# Realizar a busca usando collection.query
query_embedding_1 = model.encode(["Quais tipos de busca informada existem?"])
query_embedding_2 = model.encode(["Quais os problemas da busca exaustiva?"])

results_1 = collection.query(query_embeddings=query_embedding_1, n_results=1)
print(results_1)

results_2 = collection.query(query_embeddings=query_embedding_2, n_results=1)
print(results_2)

{'ids': [['doc_0']], 'embeddings': None, 'documents': [['busco informar diferentemente busca exaustivo onde saber bom nó fronteira expander busca heurístico estimo bom nó fronteira expander base fune']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None]], 'distances': [[0.8819772005081177]]}
{'ids': [['doc_6']], 'embeddings': None, 'documents': [['através funo heurístico estratégia sofrer mesmo problema busca exaustivo cega profundidade fazer completo pois poder ocorrer loop exemplo ótimo']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None]], 'distances': [[0.774673581123352]]}


In [114]:
# Imprimir os resultados da primeira query
print("=== Resultados da Query 1: Quais tipos de busca informada existem? ===\n")
for i in range(len(results_1['ids'][0])):
    doc_id = results_1['ids'][0][i]
    distance = results_1['distances'][0][i]
    document = results_1['documents'][0][i]
    print(f"ID: {doc_id}")
    print(f"Distância: {distance}")
    print(f"Documento: {document}")
    print("-" * 40)

print("\n=== Resultados da Query 2: Quais os problemas da busca exaustiva? ===\n")
for i in range(len(results_2['ids'][0])):
    doc_id = results_2['ids'][0][i]
    distance = results_2['distances'][0][i]
    document = results_2['documents'][0][i]
    print(f"ID: {doc_id}")
    print(f"Distância: {distance}")
    print(f"Documento: {document}")
    print("-" * 40)


=== Resultados da Query 1: Quais tipos de busca informada existem? ===

ID: doc_0
Distância: 0.8819772005081177
Documento: busco informar diferentemente busca exaustivo onde saber bom nó fronteira expander busca heurístico estimo bom nó fronteira expander base fune
----------------------------------------

=== Resultados da Query 2: Quais os problemas da busca exaustiva? ===

ID: doc_6
Distância: 0.774673581123352
Documento: através funo heurístico estratégia sofrer mesmo problema busca exaustivo cega profundidade fazer completo pois poder ocorrer loop exemplo ótimo
----------------------------------------


### Avaliação:

- **Query 1:** Distância de 0.88 - boa acurácia
- **Query 2:** Distância de 0.77 - excelente acurácia

Ambas as queries retornaram trechos relevantes do PDF que responderiam corretamente as perguntas.
